Preparation

In [15]:
#after running this, no need to restart the kurnel when you modify other files.
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import open3d
from settings import DATA_PATH
import numpy as np
from utils.visualizer import save_camera_params
import utils.visualizer as visualizer

data_type_name_pcd = "LIVOX_Hallway_pcds"
experiment_name = "jogging_fast_4th"
dir_name = "res100ms_start13.5s"

#directory path of pcd files
directory_path = os.path.join(DATA_PATH, data_type_name_pcd, experiment_name, dir_name)

#add save_path
experiment_name_eliminated = experiment_name + "_eliminated"
dir_name_eliminated= dir_name + "_eliminated"
folder_eliminated = os.path.join(DATA_PATH, data_type_name_pcd, experiment_name_eliminated, dir_name_eliminated)

if not os.path.exists(folder_eliminated):
    os.makedirs(folder_eliminated)

# make the list of files
file_list = []
for filename in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, filename)):
        file_list.append(os.path.join(directory_path, filename))
        
#select your view point and close your window.
pcd = open3d.io.read_point_cloud(file_list[0])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Main

In [16]:
from plane_elimination.plane_elimination import eliminate_planes
from plane_elimination.plane_equotions_list import planes_list, bool_list #list to define plane equotions
from detection.detect import save_bbox_result

visualize = False #whether visualize the result of elimination
save = True #save to "folder_eliminated"

#process pcd by pcd
for inx, pcd_file in enumerate(file_list):
    pcd = open3d.io.read_point_cloud(pcd_file)
    cropped_pcd = eliminate_planes(pcd, planes_list=planes_list, bool_list=bool_list, visualize=visualize)
    save_path_eliminated = folder_eliminated + '/' + os.path.basename(pcd_file)
    if save:
        open3d.io.write_point_cloud(save_path_eliminated, cropped_pcd)
    

check and visualize how elimination works (option)

In [ ]:
from plane_elimination.plane_elimination import eliminate_one_plane

pcd = open3d.io.read_point_cloud(file_list[0])

#this means clop -0.00x + -0.02y + 1.00z + -2.00 < 0 <- equotion of ceiling
plane = [-0.00, -0.02, 1.00, -2.00] #cooefficients of plane equotion
smaller = True #whether < or > in plane equotion

#outliers are visualized as red points
eliminate_one_plane(pcd, plane=plane, smaller=smaller, visualize=True)


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 要求された変換操作はサポートされていません。 


PointCloud with 15326 points.

detect plane equotions by using RANSAC (option)

In [ ]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.05,
                                            ransac_n=3,
                                            num_iterations=1000)
[a, b, c, d] = plane_model
smaller = True #whether equotion is < or >

print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

points = np.asarray(pcd.points)
values = a * points[:, 0] + b * points[:, 1] + c * points[:, 2] + d


# Select points fulfilling the equation of the plane
if smaller:
    inliers = np.where(values < 0)[0]
else:
    inliers = np.where(values > 0)[0]

# Visualize the selected points
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
outlier_cloud.paint_uniform_color([1.0, 0, 0])

open3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

Plane equation: -0.00x + -0.02y + 1.00z + 1.04 = 0
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: ハンドルが無効です。 
